In [1]:
%matplotlib inline
import tellurium as te
import numpy as np
import lmfit   # Fitting lib
import math
import random 
import matplotlib.pyplot as plt
from model_runner import ModelRunner

In [2]:
model_str1 = """
     A -> B; k1*A
      
     A = 50; 
     B = 0;
     C = 0;
     D = 0;
     k1 = 0.15
     k2 = 0.2
     k3 = 0.1
"""

timeToSimulate = 30
nDataPoints = 10

In [3]:
model_str = """
     A -> B; k1*A
     B -> C; k2*B
     B -> D; k3*B
      
     A = 50; 
     B = 0;
     C = 0;
     D = 0;
     k1 = 0.15
     k2 = 0.2
     k3 = 0.1
"""

timeToSimulate = 30
nDataPoints = 10

In [19]:
model_str2 = """
     A -> B; k1*A
     B -> C; k2*B
     C -> D; k3*B
      
     A = 50; 
     B = 0;
     C = 0;
     D = 0;
     k1 = 0.1
     k2 = 0.2
     k3 = 0.3
"""

timeToSimulate = 30
nDataPoints = 10

In [4]:
runner = ModelRunner(model_str1, ['k1'],
                    timeToSimulate, nDataPoints)

In [5]:
runner.ser_times

0     0.000000
1     3.333333
2     6.666667
3    10.000000
4    13.333333
5    16.666667
6    20.000000
7    23.333333
8    26.666667
9    30.000000
Name: time, dtype: float64

In [6]:
runner.generateObservations()

,A,B
0,49.126608,0.049277
1,29.972213,18.961095
2,17.121881,31.574006
3,10.539252,38.819605
4,6.366214,43.952503
5,4.867598,46.256012
6,2.389314,46.703631
7,0.727438,47.891914
8,1.395136,49.448993
9,0.148624,49.452802


In [7]:
# BUG
runner.fit(count=20)

,mean,std
k1,0.16028,0.064226


In [15]:
runner = ModelRunner(model_str, ['k1', 'k2', 'k3'],
                    timeToSimulate, nDataPoints)
runner.fit(count=20)

,mean,std
k1,2.471507,3.797081
k2,6.325157,3.974633
k3,3.206702,2.317920


In [20]:
runner = ModelRunner(model_str2, ['k1', 'k2', 'k3'],
                    timeToSimulate, nDataPoints)
runner.fit(count=100)

,mean,std
k1,0.108358,0.062121
k2,1.264646,2.106791
k3,2.186403,3.418765


# Running a minimization
1. Define parameters present
1. Define python function that calculates residuals given parameter values
1. Construct the minimizer
1. Run the minimizer

In [8]:
# Define the parameters present
parameters = lmfit.Parameters()
parameters.add('k1', value=1, min=0, max=10)
parameters.add('k2', value=1, min=0, max=10)
parameters.add('k3', value=1, min=0, max=10)

In [9]:
# Define a python function that computes residuals from parameter values
def residuals(p):
    rr.reset()  
    pp = p.valuesdict()
    rr.k1 = pp['k1']
    rr.k2 = pp['k2']
    rr.k2 = pp['k3']
    #data = rr.simulate (0, timeToSimulate, nDataPoints, ['B'])
    sim_data = rr.simulate (0, timeToSimulate, nDataPoints, ['A', 'B', 'C'])
    return sum(data[:,1:] - sim_data[:,:]) 


In [10]:
# Create the minimizer
fitter = lmfit.Minimizer(residuals, parameters)

In [11]:
# Run the minimizer and examine the results
result = fitter.minimize (method='leastsq')
print (result.message)

# You can also print
print (result.chisqr)
print (result.params)


NameError: name 'rr' is not defined

In [ ]:
result.params.get('k1').value

In [ ]:
def runSim(count, method='leastsq', noise_std=0.5, residuals=residuals):
    # Define the parameters present
    global y_data
    constants = {'k1': [], 'k2': [], 'k3': []}
    for _ in range(count):
        for i in range (nDataPoints):
            y_data[i] = max(data[i, 2] + np.random.normal (0, 5), 0) # standard deviation of noise
        parameters = lmfit.Parameters()
        for constant in constants.keys():
            parameters.add(constant, value=1, min=0, max=10)
        # Create the minimizer
        import pdb; pdb.set_trace()
        fitter = lmfit.Minimizer(residuals, parameters)
        result = fitter.minimize (method=method)
        for constant in constants.keys():
           constants[constant].append(result.params.get(constant).value)
    return {k: [np.mean(v), np.std(v)] for k,v in constants.items()}
        

In [ ]:
runSim(10)

## Exercises
1. Run the fit 5 times and compute the standard deviation of the parameter estimate. How does it compare with the +/- values reported by the minimizer?
1. Add the reaction B -> C; k2\*B. Redo the fit.

In [ ]:
import numpy as np
np.std([.169, .109, .139, .147, .162])